In [1]:
# Boilerplate: loading input

prog_inp = ""
with open("day4_input.txt", "r") as f:
    prog_inp = f.readlines()

prog_inp = [line.rstrip() for line in prog_inp]
print(prog_inp[:5])

['ecl:#eef340 eyr:2023 hcl:#c0946f pid:244684338 iyr:2020 cid:57 byr:1969 hgt:152cm', '', 'pid:303807545 cid:213 ecl:gry hcl:#fffffd', 'eyr:2038 byr:1951', 'hgt:171cm iyr:2011']


In [2]:
# Parsing data into usable form

passports = []
passport = []
for i, line in enumerate(prog_inp):
    if line == '':
        passports.append(passport)
        passport = []
    else:
        passport += line.split(' ')
if len(passport) > 0: passports.append(passport)

for (i, passport) in enumerate(passports):
    parsed_passport = {}
    for (j, field) in enumerate(passport):
        field = field.split(':')
        parsed_passport[field[0]] = field[1]
    passports[i] = parsed_passport

print(passports[:5])

[{'ecl': '#eef340', 'eyr': '2023', 'hcl': '#c0946f', 'pid': '244684338', 'iyr': '2020', 'cid': '57', 'byr': '1969', 'hgt': '152cm'}, {'pid': '303807545', 'cid': '213', 'ecl': 'gry', 'hcl': '#fffffd', 'eyr': '2038', 'byr': '1951', 'hgt': '171cm', 'iyr': '2011'}, {'hcl': '#c0946f', 'byr': '1933', 'eyr': '2025', 'pid': '517067213', 'hgt': '173cm', 'ecl': 'hzl', 'iyr': '2018'}, {'pid': '5253256652', 'byr': '2009', 'hgt': '152cm', 'iyr': '1989', 'eyr': '1968', 'hcl': '64cb63', 'ecl': 'hzl'}, {'iyr': '2013', 'pid': '862607211', 'eyr': '2020', 'hgt': '174cm', 'byr': '1990', 'ecl': 'blu', 'hcl': '#888785'}]


In [3]:
# This is how I did it in the competition (rushed for time)
"""
Postmortem:

For pt 1, I decided counting invalid passports was easier. However, I used `continue` instead of `break` and this bug caused my to over count invalid passports. Unable to debug this, I hacked in a counter for valid passwords and solved it that way.

For pt 2, I followed the same hacked-together mess I had in pt 1 and made a Frankenstien's monster that works. It is by far the worst thing I have written (my code from the first day of learning python is better than this) but oh well.
"""

# Field name : Required
fields = {
    "byr" : True,
    "iyr" : True,
    "eyr" : True,
    "hgt" : True,
    "hcl" : True,
    "ecl" : True,
    "pid" : True,
    "cid" : False
}

invalid_passports = 0
valid_passports = 0
for (i, passport) in enumerate(passports):
    valid = True
    for key in fields.keys():
        if fields[key] and not key in passport.keys():
            valid = False
            invalid_passports += 1
            continue
        if key == 'byr':
            try:
                yr = int(passport[key])
            except:
                valid = False
                invalid_passports += 1
                continue
            if yr < 1920 or yr > 2002:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'iyr':
            try:
                yr = int(passport[key])
            except:
                valid = False
                invalid_passports += 1
                continue
            if yr < 2010 or yr > 2020:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'eyr':
            try:
                yr = int(passport[key])
            except:
                valid = False
                invalid_passports += 1
                continue
            if yr < 2020 or yr > 2030:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'hgt':
            try:
                ht = int(passport[key][:-2])
                unit = passport[key][-2:]
            except:
                valid = False
                invalid_passports += 1
                continue
            if unit == 'in':
                if ht < 59 or ht > 76:
                    valid = False
                    invalid_passports += 1
                    continue
            elif unit == 'cm':
                if ht < 150 or ht > 193:
                    valid = False
                    invalid_passports += 1
                    continue
            else:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'hcl':
            if passport[key][0] != '#' or len(passport[key]) != 7:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'ecl':
            if not passport[key] in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']:
                valid = False
                invalid_passports += 1
                continue
        elif key == 'pid':
            if len(passport[key]) != 9:
                valid = False
                invalid_passports += 1
                continue
        
    if valid:
        valid_passports += 1

# valid_passports = len(passports) - invalid_passports
print(invalid_passports)
print(valid_passports)
print(len(passports))

409
123
257


In [12]:
# This is how I would do it if I had time (readable & maintainable, not performance optimized)

def parse_int(val, if_fail=False):
    try:
        result = int(val)
    except:
        return if_fail
    return result

def hgt_valid(val):
    if type(val) != str:
        return False
    elif len(val) <= 2:
        return False

    unit = val[-2:]
    nr = parse_int(val[:-2])

    acceptable_range = [None, None]
    if unit == 'cm': acceptable_range = [150, 193]
    elif unit == 'in': acceptable_range = [59, 76]
    else: return False

    return acceptable_range[0] <= nr <= acceptable_range[1]

def hcl_valid(val):
    if type(val) == str:
        if len(val) == 7:
            return val[0] == '#'
    return False

# Field name : Function returning whether or not the value meets the requirements for the field
# Should ALWAYS return True or False no matter the input
fields = {
    "byr" : lambda v: 1920 <= parse_int(v) <= 2002,
    "iyr" : lambda v: 2010 <= parse_int(v) <= 2020,
    "eyr" : lambda v: 2020 <= parse_int(v) <= 2030,
    "hgt" : hgt_valid,
    "hcl" : hcl_valid,
    "ecl" : lambda v: v in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'],
    "pid" : lambda v: len(v) == 9 if type(v) == str else False,
    "cid" : lambda v: True
}

invalid_passports = 0
valid_passports = 0
for (i, passport) in enumerate(passports):
    for key in fields.keys():
        if key in passport.keys():
            value = passport[key]
        else: 
            value = None
        if fields[key](value) == False:
            invalid_passports += 1
            break

valid_passports = len(passports) - invalid_passports

print(f"valid_passports: {valid_passports}")


valid_passports: 131
